In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
import torch.nn.functional as F
import numpy as np
import cv2
import numpy.linalg
plt.ion()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [2]:
from xml.etree.ElementTree import parse,dump,Element
tree = parse("./data/ss_search_image.xml")
ss_region = tree.getroot()
aaa=parse("./data/ground_truth_after_scale.xml")
gt_box=aaa.getroot()

In [3]:
#string data 를 array 로 바꾼다.
image_ss_region=[]
image_gt_box=[]
#seletive search region string to array transform
for j in range(0,17125):
    
    positive_data=[]
    for i in range(1000):
        tmp=ss_region.findtext("t_image%d/t_positive/t_%d"%(j,i))
        if tmp !=None:
            positive_data.append(tmp)
        else:
            break
    pos=[]
    neg=[]
    for i in positive_data:
        a=i
        a = a.replace("[", "")
        a = a.replace("]", "")
        a = a.replace(" ", "")
        b=[]
        words = a.split(",")
        for i in words:
            b.append(int(i))
        pos.append(b)
    image_ss_region.append(pos)
#ground truth region string to array transform
for j in range(0,17125):
    tmp=gt_box.findtext("image_%d/truth_box"%j)
    tmp = tmp.replace("[", "")
    tmp = tmp.replace("]", "")
    tmp = tmp.replace(" ", "")
    b=[]
    words=tmp.split(",")
    for i in words:
        b.append(int(i))
    image_gt_box.append(b)

In [4]:
#500개의 이미지의 특징들을 가져옴
import glob
from PIL import Image

# img_part=1
cnt=0
image_data=[]
# for img_part in range(1,len(total_part)+1):#file directory전부를 불러온다.
for img_part in range(1,2):#file directory
    img_cnt=glob.glob("D:/sppnet_feature2/%d/*.jpg"%img_part)
    img_num=int(len(img_cnt)/256)
    print(img_part,len(img_cnt),img_num,cnt)
#     for i in range(cnt,cnt+img_num):# image number
    for i in range(cnt,1):# image number
#         print(i)
        for j in range(256):#feature number
            st="D:/sppnet_feature2/%d/image%d_feature%d.jpg"%(img_part,i,j)
#             print(st)
            n=Image.open(st)
#             n=cv2.cvtColor(n, cv2.COLOR_BGR2RGB)# matplotlib = RGB channel , opencv = BGR channels
            image_data.append(n)
    cnt=cnt+img_num


1 128000 500 0


In [5]:
print("ground truth box 갯수 : ",len(image_gt_box))
print("관심영역의 갯수 : ",len(image_ss_region[0]))
print("이미지 1의 특징의 갯수 : ",len(image_data))

ground truth box 갯수 :  17125
관심영역의 갯수 :  7
이미지 1의 특징의 갯수 :  256


In [6]:
print(image_gt_box[0])

[5, 2, 10, 9]


In [7]:
#크롭영역의 크기 resize 연산오류 나지 않도록
from PIL import Image
cnt=0
for pic in range(256):
#     print(len(image_ss_region[pic]))
    for i in range(len(image_ss_region[pic])):
        image=image_data[pic]#image_data
        ss=image_ss_region[pic][i]#positive example의 갯수만큼 반복한다.
        w = abs(ss[2] - ss[0])
        h = abs(ss[3] - ss[1])
        area=(ss[0],ss[1],ss[0]+w,ss[1]+h)
        im=image.crop(area)
        size=(im.size[0]+20,im.size[1]+20)
        if(size[0]>0 or size[1]>0):
#             print(cnt, size)
            im2=im.resize(size,Image.ANTIALIAS)
            im2.save("./data/crop_image/scaled_%d.jpg"%cnt)
        cnt+=1
print(cnt)


# print(img.size)
# im_w=img.size[0]*10
# im_h=img.size[1]*10
# i2=img.resize((im_w,im_h),Image.ANTIALIAS)
# print(i2.size)
# plt.imshow(img)


# # #cv2 version
# # #pytorch 에서 불러올 데이터를 저장한다.
# pic=0
# i=0
# cnt=0
# for pic in range(256):
# #     print(len(image_ss_region[pic]))
#     for i in range(len(image_ss_region[pic])):
#         image=image_data[pic]#image_data
#         ss=image_ss_region[pic][i]#positive example의 갯수만큼 반복한다.
#         w = abs(ss[2] - ss[0])
#         h = abs(ss[3] - ss[1])
#         im=image[ss[0] : ss[0]+w,ss[1] : ss[1]+h]
#         cv2.imwrite("./data/crop_image/scaled_%d.jpg"%cnt,im)
#         cnt+=1
# print(cnt)

1934


In [8]:

#크기가 0인 이미지를 제거하는 방법
def delete_size_zero(length):
    import os
    cnt=0
    for i in range(length):
        strr="data/crop_image/scaled_%d.jpg"%i
        f_size=os.path.getsize(strr)
        if (f_size==0):
#             print(i,f_size)
            os.remove(strr)
            cnt+=1
    print(cnt)
#크기가 0인 이미지를 지운다.
delete_size_zero(1934)

0


In [9]:
#SPP net 
a=int(14)
class SpatialPooling(nn.Module):
    def __init__(self):
        super(SpatialPooling,self).__init__()
        a=int(14)
        spp6_ksize=round(a/6+0.49)
        spp6_stride=int(a/6)
        self.spp6_layer=nn.MaxPool2d(kernel_size=spp6_ksize,
                                    stride=spp6_stride,padding=0)
        spp3_ksize=round(a/3+0.49)
        spp3_stride=int(a/3)
        self.spp3_layer=nn.MaxPool2d(kernel_size=spp3_ksize,
                                    stride=spp3_stride,padding=0)
        spp2_ksize=round(a/2+0.49)
        spp2_stride=int(a/2)
        self.spp2_layer=nn.MaxPool2d(kernel_size=spp2_ksize,
                                    stride=spp2_stride,padding=0)
        spp1_ksize=round(a/1+0.49)
        spp1_stride=int(a/1)
        self.spp1_layer=nn.MaxPool2d(kernel_size=spp1_ksize,
                                    stride=spp1_stride,padding=0)
        
        
    def forward(self,x):
        out1=self.spp6_layer(x)
        out2=self.spp3_layer(x)
        out3=self.spp2_layer(x)
        out4=self.spp1_layer(x)
        print(out1.shape,out2.shape,out3.shape,out4.shape)
        out1 = out1.reshape(out1.size(0), -1)
        out2 = out2.reshape(out2.size(0), -1)
        out3 = out3.reshape(out3.size(0), -1)
        out4 = out4.reshape(out4.size(0), -1)

        print(out1.shape,out2.shape,out3.shape,out4.shape)
        
        out=torch.cat((out1,out2,out3,out4),dim=1)
        
        print("concat : ",out.shape)
        
        in_len=int(out1.size(1)+out2.size(1)+out3.size(1)+out4.size(1))
        self.fc1=nn.Linear(in_len,1024)
        self.fc2=nn.Linear(1024,4)
        out=self.fc1(out)
        out=self.fc2(out)
        return out 

In [10]:
##bounding box regression 옵션##

min_overlap=0.5
lamda=4000


In [11]:
#특징의 fc거친값 , target 값을 만드는 함수
def get_examples(subim,gt_box,ex_box):
    sp=SpatialPooling().to(device)
#     subim=torch.from_numpy(subim)
    x=sp.forward(subim).to(device)
    #img의 x1,y1,x2,y2순으로 되어있다.
    
    src_w=ex_box[2]-ex_box[0]
    src_h=ex_box[3]-ex_box[1]
    
    src_ctr_x=ex_box[0]+0.5*src_w
    src_ctr_y=ex_box[1]+0.5*src_h
    
    gt_w=gt_box[2]-gt_box[0]
    gt_h=gt_box[3]-gt_box[1]
    gt_ctr_x=gt_box[0]+0.5*gt_w
    gt_ctr_y=gt_box[1]+0.5*gt_h
    
    if(src_w!=0 and src_h!=0):
        dst_ctr_x=(gt_ctr_x-src_ctr_x)*1/src_w
        dst_ctr_y=(gt_ctr_y-src_ctr_y)*1/src_h
        dst_scl_w=np.log(gt_w/src_w)
        dst_scl_h=np.log(gt_h/src_h)
        target=[dst_ctr_x,dst_ctr_y,dst_scl_w,dst_scl_h]
    else:
        dst_ctr_x=0
        dst_ctr_y=0
        dst_scl_w=0
        dst_scl_h=0
        target=[dst_ctr_x,dst_ctr_y,dst_scl_w,dst_scl_h]
    y=target
    return x,y

    

In [12]:
#크롭된 이미지를 불러와서 데이터로 정렬
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
crop_image=torchvision.datasets.ImageFolder("data",transforms.Compose([
                                            transforms.ToTensor()]))
crop_loader=torch.utils.data.DataLoader(dataset=crop_image,
                                         batch_size=1,
                                         shuffle=False)
print(crop_image,crop_loader)

Dataset ImageFolder
    Number of datapoints: 1934
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None <torch.utils.data.dataloader.DataLoader object at 0x000001A1656A2BA8>


In [13]:
print(len(image_ss_region))
print(image_ss_region[0])

17125
[[4, 0, 9, 10], [0, 8, 3, 4], [4, 0, 8, 7], [4, 0, 8, 8], [4, 0, 9, 8], [4, 0, 9, 8], [2, 0, 9, 9]]


In [14]:
crop_imm=[]
for i,(crop_im,labels) in enumerate(crop_loader):
    crop_imm.append(crop_im)
print(len(crop_imm))

1934


In [15]:
#get examples 을 통해서 특징 및 타겟을 받아옴 

X=[]# 특징 모음
Y=[]# Target 모음
#특징을 추출하여 Fc를 거쳐서 추측하는 4개값 만든다. target 의 값도 같이 가져온다.
# for i,(crop_im,labels) in enumerate(crop_loader):
#     crop_im.to(device)
#     for image_num in range(500):
#         for j in range(len(image_ss_region[image_num])):
#             print("crop_im : ",i,"gt_box : ",image_num,"ss_region : ",j)

#이미지 한개 에 특징 256개 그 특징의 관심영역이 1933개
crop_num=0
for feat_num in range(256):
    for ss_num in range(len(image_ss_region[feat_num])):
#         print("gt_box : ",feat_num,", crop_im : ",crop_num,", ss_region : ",ss_num)
        
        crop_imm[crop_num].to(device)
        X_data,Y_data=get_examples(crop_imm[crop_num],image_gt_box[feat_num],image_ss_region[feat_num][ss_num])
        X.append(X_data)
        Y.append(Y_data)
        crop_num+=1

torch.Size([1, 3, 14, 12]) torch.Size([1, 3, 7, 6]) torch.Size([1, 3, 4, 3]) torch.Size([1, 3, 2, 1])
torch.Size([1, 504]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 672])
torch.Size([1, 3, 11, 11]) torch.Size([1, 3, 5, 5]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 363]) torch.Size([1, 75]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 468])
torch.Size([1, 3, 10, 15]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 450]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 597])
torch.Size([1, 3, 13, 13]) torch.Size([1, 3, 6, 6]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 507]) torch.Size([1, 108]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 657])
torch.Size([1, 3, 11, 11]) torch.Size([1, 3, 5, 5]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 363]) torch.Size([1, 75]) to

C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in log


 torch.Size([1, 3, 7, 9]) torch.Size([1, 3, 4, 5]) torch.Size([1, 3, 2, 2])
torch.Size([1, 810]) torch.Size([1, 189]) torch.Size([1, 60]) torch.Size([1, 12])
concat :  torch.Size([1, 1071])
torch.Size([1, 3, 14, 15]) torch.Size([1, 3, 7, 7]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 630]) torch.Size([1, 147]) torch.Size([1, 48]) torch.Size([1, 12])
concat :  torch.Size([1, 837])
torch.Size([1, 3, 12, 14]) torch.Size([1, 3, 6, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 504]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 672])
torch.Size([1, 3, 15, 15]) torch.Size([1, 3, 7, 7]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 675]) torch.Size([1, 147]) torch.Size([1, 48]) torch.Size([1, 12])
concat :  torch.Size([1, 882])
torch.Size([1, 3, 12, 18]) torch.Size([1, 3, 6, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 648]) torch.Size([1, 162]) torch.Size([1, 45]) to

torch.Size([1, 3, 11, 15]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 495]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 642])
torch.Size([1, 3, 11, 18]) torch.Size([1, 3, 5, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 594]) torch.Size([1, 135]) torch.Size([1, 45]) torch.Size([1, 6])
concat :  torch.Size([1, 780])
torch.Size([1, 3, 12, 19]) torch.Size([1, 3, 6, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 684]) torch.Size([1, 162]) torch.Size([1, 45]) torch.Size([1, 6])
concat :  torch.Size([1, 897])
torch.Size([1, 3, 12, 15]) torch.Size([1, 3, 6, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 540]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 708])
torch.Size([1, 3, 12, 17]) torch.Size([1, 3, 6, 8]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 612]) torch.Size([1, 144]) 

torch.Size([1, 3, 13, 11]) torch.Size([1, 3, 6, 5]) torch.Size([1, 3, 4, 3]) torch.Size([1, 3, 2, 1])
torch.Size([1, 429]) torch.Size([1, 90]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 561])
torch.Size([1, 3, 16, 15]) torch.Size([1, 3, 8, 7]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 720]) torch.Size([1, 168]) torch.Size([1, 48]) torch.Size([1, 12])
concat :  torch.Size([1, 948])
torch.Size([1, 3, 13, 16]) torch.Size([1, 3, 6, 8]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 624]) torch.Size([1, 144]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 810])
torch.Size([1, 3, 16, 16]) torch.Size([1, 3, 8, 8]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 768]) torch.Size([1, 192]) torch.Size([1, 48]) torch.Size([1, 12])
concat :  torch.Size([1, 1020])
torch.Size([1, 3, 15, 15]) torch.Size([1, 3, 7, 7]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 675]) torch.Size([1, 147]

...

torch.Size([1, 3, 15, 13]) torch.Size([1, 3, 7, 6]) torch.Size([1, 3, 4, 3]) torch.Size([1, 3, 2, 1])
torch.Size([1, 585]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 753])
torch.Size([1, 3, 12, 14]) torch.Size([1, 3, 6, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 504]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 672])
torch.Size([1, 3, 13, 15]) torch.Size([1, 3, 6, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 585]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 753])
torch.Size([1, 3, 12, 14]) torch.Size([1, 3, 6, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 504]) torch.Size([1, 126]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 672])
torch.Size([1, 3, 14, 18]) torch.Size([1, 3, 7, 9]) torch.Size([1, 3, 4, 5]) torch.Size([1, 3, 2, 2])
torch.Size([1, 756]) torch.Size([1, 189]) 

torch.Size([1, 468]) torch.Size([1, 108]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 606])
torch.Size([1, 3, 11, 12]) torch.Size([1, 3, 5, 6]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 396]) torch.Size([1, 90]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 516])
torch.Size([1, 3, 10, 11]) torch.Size([1, 3, 5, 5]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 330]) torch.Size([1, 75]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 435])
torch.Size([1, 3, 14, 13]) torch.Size([1, 3, 7, 6]) torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 2, 2])
torch.Size([1, 546]) torch.Size([1, 126]) torch.Size([1, 48]) torch.Size([1, 12])
concat :  torch.Size([1, 732])
torch.Size([1, 3, 16, 11]) torch.Size([1, 3, 8, 5]) torch.Size([1, 3, 4, 3]) torch.Size([1, 3, 2, 1])
torch.Size([1, 528]) torch.Size([1, 120]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 690])
torch.Size([1, 3, 15, 11]) torch.

torch.Size([1, 3, 10, 14]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 420]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 567])
torch.Size([1, 3, 10, 14]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 420]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 567])
torch.Size([1, 3, 10, 14]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 420]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 567])
torch.Size([1, 3, 10, 14]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 420]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 567])
torch.Size([1, 3, 10, 16]) torch.Size([1, 3, 5, 8]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 480]) torch.Size([1, 120]) 

torch.Size([1, 3, 12, 12]) torch.Size([1, 3, 6, 6]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 432]) torch.Size([1, 108]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 570])
torch.Size([1, 3, 12, 19]) torch.Size([1, 3, 6, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 684]) torch.Size([1, 162]) torch.Size([1, 45]) torch.Size([1, 6])
concat :  torch.Size([1, 897])
torch.Size([1, 3, 12, 19]) torch.Size([1, 3, 6, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 684]) torch.Size([1, 162]) torch.Size([1, 45]) torch.Size([1, 6])
concat :  torch.Size([1, 897])
torch.Size([1, 3, 13, 19]) torch.Size([1, 3, 6, 9]) torch.Size([1, 3, 3, 5]) torch.Size([1, 3, 1, 2])
torch.Size([1, 741]) torch.Size([1, 162]) torch.Size([1, 45]) torch.Size([1, 6])
concat :  torch.Size([1, 954])
torch.Size([1, 3, 14, 19]) torch.Size([1, 3, 7, 9]) torch.Size([1, 3, 4, 5]) torch.Size([1, 3, 2, 2])
torch.Size([1, 798]) torch.Size([1, 189]) 

torch.Size([1, 3, 9, 16]) torch.Size([1, 3, 4, 8]) torch.Size([1, 3, 2, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 432]) torch.Size([1, 96]) torch.Size([1, 24]) torch.Size([1, 6])
concat :  torch.Size([1, 558])
torch.Size([1, 3, 12, 13]) torch.Size([1, 3, 6, 6]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 468]) torch.Size([1, 108]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 618])
torch.Size([1, 3, 14, 10]) torch.Size([1, 3, 7, 5]) torch.Size([1, 3, 4, 3]) torch.Size([1, 3, 2, 1])
torch.Size([1, 420]) torch.Size([1, 105]) torch.Size([1, 36]) torch.Size([1, 6])
concat :  torch.Size([1, 567])
torch.Size([1, 3, 13, 11]) torch.Size([1, 3, 6, 5]) torch.Size([1, 3, 3, 3]) torch.Size([1, 3, 1, 1])
torch.Size([1, 429]) torch.Size([1, 90]) torch.Size([1, 27]) torch.Size([1, 3])
concat :  torch.Size([1, 549])
torch.Size([1, 3, 11, 15]) torch.Size([1, 3, 5, 7]) torch.Size([1, 3, 3, 4]) torch.Size([1, 3, 1, 2])
torch.Size([1, 495]) torch.Size([1, 105]) tor

In [ ]:
#mu, T, T_inv 를 전송 -->solve robust 로
# Yi을 열 단위로 solve robust 한다
#A= x.y=Yi, lamda=4000
#loss function
def solve(A,y,lamda):
# #   R = chol(A*A' + lambda*eye(size(A,1)));
#     R=np.linalg.cholesky(np.dot(A,np.transpose(A))+lamda*np.eye(np.size(A,1)))
# #     R=np.linalg.cholesky(np.dot(A,np.transpose(A)))
    
#     z=np.linalg.lstsq(np.transpose(R),np.transpose(A)*y)
#     print("R shape : ",np.shape(R)," ,z shape : ",np.shape(z))
    
#     x=np.linalg.lstsq(R,z)

#     x = np.linalg.inv(np.dot(np.transpose(A)*A) +lamda*np.identity(size(A,1)))*(np.dot(np.transpose(A),y))
    x = np.linalg.lstsq(np.transpose(A),y)
    losses=0.5*((np.transpose(A)*x-y)**2)
    
    return x,losses
def solve_robust(A,y,lamda):
    [x,losses]=solve(A,y,lamda)
    print("losses : ",np.mean(losses))


In [16]:

import numpy.matlib
def bsxfun(case,mat1,mat2):
    remat1=np.matlib.repmat(mat1,len(mat1),1)
    remat2=np.matlib.repmat(mat2,len(mat1),1)
    remat2=np.transpose(remat2)
    if(case=="plus"):
        return remat1+remat2
    elif(case=="minus"):
        return remat1-remat2

In [17]:
Yi=[]
for i in range(len(Y)):
    # x,y 데이터 나옴
    mu=np.mean(Y[i]) # 평균값
    Yi.append(bsxfun("minus",Y[i],mu))
for i in range(len(Yi)):
    Yi[i]=np.nan_to_num(Yi[i])
    Yi[i]=np.float32(Yi[i])    

#     print(Yi)

C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in subtract
  # Remove the CWD from sys.path while we load stuff.


In [185]:
for i in range(len(Yi)):
    S=np.dot(np.transpose(Yi[i]),Yi[i])/np.size(Yi[i],1)
    s = np.nan_to_num(S)
    S=s
    #고유값과 고유벡터 받는다.
    # [V, D] = eig(S);
    V = np.linalg.eig(S)[0] #벡터
    D = np.linalg.eig(S)[1] #행렬
    #  D = diag(D);
    D=np.diag(D)

    T=V*np.diag(1./np.sqrt(D+0.001))*np.transpose(V)
    T_inv=V*np.diag(np.sqrt(D+0.001))*np.transpose(V)

    Yi[i]=Yi[i]*T
    X[i]=(X[i].data).cpu().numpy()
    solve_robust(X[i],Yi[i][:,0],lamda)
    solve_robust(X[i],Yi[i][:,1],lamda)
    solve_robust(X[i],Yi[i][:,2],lamda)
    solve_robust(X[i],Yi[i][:,3],lamda)

C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  app.launch_new_instance()
C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply


losses :  [0.00037962]
losses :  [0.00037962]
losses :  [0.00037962]
losses :  [0.00037962]
losses :  [0.00067005]
losses :  [0.00067005]
losses :  [0.00067005]
losses :  [0.00067005]
losses :  [0.00614809]
losses :  [0.00614809]
losses :  [0.00614809]
losses :  [0.00614809]
losses :  [0.00197455]
losses :  [0.00197455]
losses :  [0.00197455]
losses :  [0.00197455]
losses :  [0.00013251]
losses :  [0.00013251]
losses :  [0.00013251]
losses :  [0.00013251]
losses :  [0.00093185]
losses :  [0.00093185]
losses :  [0.00093185]
losses :  [0.00093185]
losses :  [0.00217128]
losses :  [0.00217128]
losses :  [0.00217128]
losses :  [0.00217128]
losses :  [0.00117359]
losses :  [0.00117359]
losses :  [0.00117359]
losses :  [0.00117359]
losses :  [0.00345508]
losses :  [inf]
losses :  [0.00345508]
losses :  [0.00345508]
losses :  [0.00021293]
losses :  [inf]
losses :  [0.00021293]
losses :  [0.00021293]
losses :  [0.00027682]
losses :  [inf]
losses :  [0.00027682]
losses :  [0.00027682]
losses : 

losses :  [0.00259615]
losses :  [0.00259615]
losses :  [0.00259615]
losses :  [0.00259615]
losses :  [0.00245173]
losses :  [0.00245173]
losses :  [0.00245173]
losses :  [0.00245173]
losses :  [0.00089735]
losses :  [0.00089735]
losses :  [0.00089735]
losses :  [0.00089735]
losses :  [0.00132657]
losses :  [0.00132657]
losses :  [0.00132657]
losses :  [0.00132657]
losses :  [0.00236935]
losses :  [0.00236935]
losses :  [0.00236935]
losses :  [0.00236935]
losses :  [0.00232129]
losses :  [0.00232129]
losses :  [0.00232129]
losses :  [0.00232129]
losses :  [0.00645884]
losses :  [0.00645884]
losses :  [0.00645884]
losses :  [0.00645884]
losses :  [0.00047922]
losses :  [0.00047922]
losses :  [0.00047922]
losses :  [0.00047922]
losses :  [0.00058527]
losses :  [0.00058527]
losses :  [0.00058527]
losses :  [0.00058527]
losses :  [0.00120911]
losses :  [0.00120911]
losses :  [0.00120911]
losses :  [0.00120911]
losses :  [0.00452274]
losses :  [0.00452274]
losses :  [0.00452274]
losses :  [

losses :  [0.00588902]
losses :  [0.00588902]
losses :  [0.00022604]
losses :  [0.00022604]
losses :  [0.00022604]
losses :  [0.00022604]
losses :  [0.00070849]
losses :  [0.00070849]
losses :  [0.00070849]
losses :  [0.00070849]
losses :  [0.00118029]
losses :  [0.00118029]
losses :  [0.00118029]
losses :  [0.00118029]
losses :  [0.00064585]
losses :  [0.00064585]
losses :  [0.00064585]
losses :  [0.00064585]
losses :  [0.00263455]
losses :  [0.00263455]
losses :  [0.00263455]
losses :  [0.00263455]
losses :  [0.00076224]
losses :  [0.00076224]
losses :  [0.00076224]
losses :  [0.00076224]
losses :  [0.00126478]
losses :  [0.00126478]
losses :  [0.00126478]
losses :  [0.00126478]
losses :  [0.00020465]
losses :  [88.641426]
losses :  [0.00020465]
losses :  [0.00020465]
losses :  [0.00668122]
losses :  [0.00668122]
losses :  [0.00668122]
losses :  [0.00668122]
losses :  [0.00048056]
losses :  [0.00048056]
losses :  [0.00048056]
losses :  [0.00048056]
losses :  [0.0022648]
losses :  [0.

C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in multiply
  if sys.path[0] == '':
C:\Users\Robotmedia9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]


...

losses :  [0.00019787]
losses :  [0.00019787]
losses :  [0.00012197]
losses :  [0.00012197]
losses :  [0.00012197]
losses :  [0.00012197]
losses :  [4.137818e-05]
losses :  [4.137818e-05]
losses :  [4.137818e-05]
losses :  [4.137818e-05]
losses :  [0.00053877]
losses :  [0.00053877]
losses :  [0.00053877]
losses :  [0.00053877]
losses :  [0.0003237]
losses :  [0.0003237]
losses :  [0.0003237]
losses :  [0.0003237]
losses :  [0.00067066]
losses :  [0.00067066]
losses :  [0.00067066]
losses :  [0.00067066]
losses :  [0.00192823]
losses :  [0.00192823]
losses :  [0.00192823]
losses :  [0.00192823]
losses :  [0.00015677]
losses :  [0.00015677]
losses :  [0.00015677]
losses :  [0.00015677]
losses :  [0.00032435]
losses :  [0.00032435]
losses :  [0.00032435]
losses :  [0.00032435]
losses :  [0.0003906]
losses :  [0.0003906]
losses :  [0.0003906]
losses :  [0.0003906]
losses :  [3.9545805e-05]
losses :  [3.9545805e-05]
losses :  [3.9545805e-05]
losses :  [3.9545805e-05]
losses :  [9.122957e-0

losses :  [4.7707454e-05]
losses :  [0.00071664]
losses :  [0.00071664]
losses :  [0.00071664]
losses :  [0.00071664]
losses :  [4.266311e-05]
losses :  [4.266311e-05]
losses :  [4.266311e-05]
losses :  [4.266311e-05]
losses :  [0.00714211]
losses :  [0.00714211]
losses :  [0.00714211]
losses :  [0.00714211]
losses :  [0.02281839]
losses :  [0.02281839]
losses :  [0.02281839]
losses :  [0.02281839]
losses :  [0.00212027]
losses :  [0.00212027]
losses :  [0.00212027]
losses :  [0.00212027]
losses :  [0.00536103]
losses :  [0.00536103]
losses :  [0.00536103]
losses :  [0.00536103]
losses :  [0.0054551]
losses :  [0.0054551]
losses :  [0.0054551]
losses :  [0.0054551]
losses :  [0.00746926]
losses :  [0.00746926]
losses :  [0.00746926]
losses :  [0.00746926]
losses :  [8.57744e-05]
losses :  [8.57744e-05]
losses :  [8.57744e-05]
losses :  [8.57744e-05]
losses :  [inf]
losses :  [0.01265976]
losses :  [0.01265976]
losses :  [0.01265976]
losses :  [inf]
losses :  [0.00628021]
losses :  [0.0

losses :  [0.00071548]
losses :  [0.00071548]
losses :  [0.00088299]
losses :  [0.00088299]
losses :  [0.00088299]
losses :  [0.00088299]
losses :  [0.00045637]
losses :  [0.00045637]
losses :  [0.00045637]
losses :  [0.00045637]
losses :  [0.00078497]
losses :  [0.00078497]
losses :  [0.00078497]
losses :  [0.00078497]
losses :  [0.00072411]
losses :  [0.00072411]
losses :  [0.00072411]
losses :  [0.00072411]
losses :  [0.00178992]
losses :  [0.00178992]
losses :  [0.00178992]
losses :  [0.00178992]
losses :  [5.0787898e-05]
losses :  [5.074584e-05]
losses :  [5.074584e-05]
losses :  [5.074584e-05]
losses :  [nan]
losses :  [0.00085286]
losses :  [0.00085286]
losses :  [0.00085286]
losses :  [0.00122298]
losses :  [0.00122298]
losses :  [0.00122298]
losses :  [0.00122298]
losses :  [0.06612668]
losses :  [0.00015703]
losses :  [0.00015703]
losses :  [0.00015703]
losses :  [0.00222435]
losses :  [0.00222435]
losses :  [0.00222435]
losses :  [0.00222435]
losses :  [4.9450846e-05]
losses

losses :  [0.0024785]
losses :  [1194.59]
losses :  [0.0024785]
losses :  [0.0024785]
losses :  [0.00899846]
losses :  [inf]
losses :  [0.00899846]
losses :  [0.00899846]
losses :  [0.0009]
losses :  [0.0009]
losses :  [0.0009]
losses :  [0.0009]
losses :  [0.00305689]
losses :  [0.00305689]
losses :  [0.00305689]
losses :  [0.00305689]
losses :  [0.00015315]
losses :  [inf]
losses :  [0.00015315]
losses :  [0.00015315]
losses :  [0.00060112]
losses :  [0.00060112]
losses :  [0.00060112]
losses :  [0.00060112]
losses :  [0.00547623]
losses :  [0.00547623]
losses :  [0.00547623]
losses :  [0.00547623]
losses :  [0.0016218]
losses :  [0.0016218]
losses :  [0.0016218]
losses :  [0.0016218]
losses :  [0.0004236]
losses :  [0.00042361]
losses :  [0.0004236]
losses :  [0.0004236]
losses :  [0.00048013]
losses :  [0.00048013]
losses :  [0.00048013]
losses :  [0.00048013]
losses :  [0.00071882]
losses :  [0.00071882]
losses :  [0.00071882]
losses :  [0.00071882]
losses :  [0.00037668]
losses :

losses :  [0.00011871]
losses :  [0.00011871]
losses :  [0.00011871]
losses :  [0.00011871]
losses :  [0.00014748]
losses :  [0.00014748]
losses :  [0.00014748]
losses :  [0.00014748]
losses :  [0.00106011]
losses :  [0.00106011]
losses :  [0.00106011]
losses :  [0.00106011]
losses :  [0.00089746]
losses :  [0.00089746]
losses :  [0.00089746]
losses :  [0.00089746]
losses :  [8.609034e-05]
losses :  [8.609034e-05]
losses :  [8.609034e-05]
losses :  [8.609034e-05]
losses :  [0.00065015]
losses :  [0.00064909]
losses :  [0.00064909]
losses :  [0.00064909]
losses :  [0.00024091]
losses :  [0.00024091]
losses :  [0.00024091]
losses :  [0.00024091]
losses :  [0.00042117]
losses :  [0.00034376]
losses :  [0.00034376]
losses :  [0.00034376]
losses :  [0.00070697]
losses :  [0.00070697]
losses :  [0.00070697]
losses :  [0.00070697]
losses :  [6.659374e-05]
losses :  [6.659374e-05]
losses :  [6.659374e-05]
losses :  [6.659374e-05]
losses :  [0.00301361]
losses :  [0.00301361]
losses :  [0.00301